In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
!pip -q install transformers
!pip -q install datasets
!pip -q install simpletransformers
!pip -q install pythainlp

     |████████████████████████████████| 4.7 MB 6.8 MB/s 
     |████████████████████████████████| 6.6 MB 46.4 MB/s 
     |████████████████████████████████| 101 kB 13.3 MB/s 
     |████████████████████████████████| 596 kB 87.6 MB/s 
     |████████████████████████████████| 365 kB 7.0 MB/s 
     |████████████████████████████████| 212 kB 79.1 MB/s 
     |████████████████████████████████| 141 kB 71.7 MB/s 
     |████████████████████████████████| 115 kB 88.2 MB/s 
     |████████████████████████████████| 127 kB 78.1 MB/s 
     |████████████████████████████████| 249 kB 8.5 MB/s 
     |████████████████████████████████| 9.1 MB 71.4 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 1.8 MB 52.6 MB/s 
     |████████████████████████████████| 1.3 MB 79.6 MB/s 
     |████████████████████████████████| 181 kB 77.0 MB/s 
     |████████████████████████████████| 157 kB 86.3 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |███████████████

In [3]:
import numpy as np
import pandas as pd

In [4]:
from datasets import load_dataset
thaiqa_ = load_dataset('thaiqa_squad')

Generating train split:   0%|          | 0/4000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/74 [00:00<?, ? examples/s]

Dataset thaiqa_squad downloaded and prepared to /root/.cache/huggingface/datasets/thaiqa_squad/thaiqa_squad/1.0.0/fce14864b511d48464540780f328f4b415746b63f2fd934ad0b06c3eead7787b. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
%load_ext autoreload
%autoreload 2

import pythainlp, transformers
pythainlp.__version__, transformers.__version__ #fix pythainlp to stabilize word tokenization for metrics
import collections
import logging
import pprint
import re
from tqdm.auto import tqdm

import numpy as np
import torch

#datasets
from datasets import (
    load_dataset, 
    load_metric, 
    concatenate_datasets,
    load_from_disk,
)


In [6]:
thaiqa_

DatasetDict({
    train: Dataset({
        features: ['question_id', 'article_id', 'context', 'question', 'answers'],
        num_rows: 4000
    })
    validation: Dataset({
        features: ['question_id', 'article_id', 'context', 'question', 'answers'],
        num_rows: 74
    })
})

In [7]:
import pandas as pd
q1 = pd.DataFrame(thaiqa_["train"])
q1 = q1.loc[:,['question_id', 'article_id', 'context', 'question', 'answers']]
q2 = pd.DataFrame(thaiqa_["validation"])
q2 = q2.loc[:,['question_id', 'article_id', 'context', 'question', 'answers']]

# q3 = pd.DataFrame(thaiqa_["train"])

In [8]:
from lxml import etree
parser = etree.XMLParser(recover=True)
tree = etree.fromstring(q1["context"][0], parser=parser)
print(tree.tag)
print(tree.attrib)
print(tree.text)

doc
{'id': '115035', 'url': 'https://th.wikipedia.org/wiki?curid=115035', 'title': 'เบนจี้'}
เบนจี้ เบนจี้ () เป็นชื่อตัวละครหมาพันทางแสนรู้ ที่ปรากฏอยู่ในภาพยนตร์หลายเรื่องที่เขียนบท และกำกับโดย โจ แคมป์ ในช่วงทศวรรษ 1970 ถึง 1980 ภาพยนตร์เรื่องแรกในชุด ใช้ชื่อเรื่องว่า เบนจี้ เช่นเดียวกับตัวละคร ถ่ายทำที่เมืองดัลลัส รัฐเทกซัส ฉายครั้งแรกในปี พ.ศ. 2517 ภาพยนตร์ได้รับการเสนอชื่อเข้าชิงรางวัลออสการ์ และได้รางวัลลูกโลกทองคำ สาขาเพลงประกอบยอดเยี่ยม จากเพลง Benji's Theme (I Feel Love) ร้องโดย ชาร์ลี ริช หมาที่แสดงเป็นเบนจี้ตัวแรก ชื่อว่า ฮิกกิ้นส์ (พ.ศ. 2502 - พ.ศ. 2518) มีอายุถึง 15 ปีแล้วในขณะแสดง หลังจากภาพยนตร์ออกฉายได้ไม่นาน มันก็ตายในปี พ.ศ. 2518เบนจี้ในภาพยนตร์เบนจี้ในภาพยนตร์. - พ.ศ. 2517, Benji (ภาพยนตร์) - พ.ศ. 2520, For the Love of Benji (ภาพยนตร์) - พ.ศ. 2521, Benji's Very Own Christmas Story (ภาพยนตร์โทรทัศน์) - พ.ศ. 2523, Oh Heavenly Dog (ภาพยนตร์) - พ.ศ. 2523, Benji at Work (ภาพยนตร์โทรทัศน์) - พ.ศ. 2524, Benji Takes a Dive at Marineland (ภาพยนตร์โทรทัศน์) - พ.ศ. 2526, Benji

In [9]:
def extract_text(body):
  tree = etree.fromstring(body, parser=parser)
  return tree.text

In [10]:
q1

,question_id,article_id,context,question,answers
0,1,115035,"<doc id=""115035"" url=""https://th.wikipedia.org...",สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Ben...,"{'answer': ['ฮิกกิ้นส์'], 'answer_begin_positi..."
1,2035,115035,"<doc id=""115035"" url=""https://th.wikipedia.org...",เพลง Benji's Theme ใช้ประกอบภาพยนตร์เรื่อง Ben...,"{'answer': ['ชาร์ลี ริช'], 'answer_begin_posit..."
2,2,376583,"<doc id=""376583"" url=""https://th.wikipedia.org...",ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,"{'answer': ['เมชตา'], 'answer_begin_position':..."
3,3,376583,"<doc id=""376583"" url=""https://th.wikipedia.org...",ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,"{'answer': ['ลมสุริยะ'], 'answer_begin_positio..."
4,4,547560,"<doc id=""547560"" url=""https://th.wikipedia.org...",กระทรวงโฆษณาแถลงข่าวและโฆษณาชวนเชื่อของนาซีเยอ...,"{'answer': ['ปี 1933'], 'answer_begin_position..."
...,...,...,...,...,...
3995,3996,5114,"<doc id=""5114"" url=""https://th.wikipedia.org/w...",จังหวัดใดมีพื้นที่มากที่สุดในประเทศไทย,"{'answer': ['นครราชสีมา'], 'answer_begin_posit..."
3996,3997,5114,"<doc id=""5114"" url=""https://th.wikipedia.org/w...",ดอกไม้ประจำจังหวัดนครราชสีมาในประเทศไทยคือดอกอะไร,"{'answer': ['ดอกสาธร'], 'answer_begin_position..."
3997,3998,20722,"<doc id=""20722"" url=""https://th.wikipedia.org/...",เมืองหลวงของประเทศโมร็อกโกมีชื่อว่าอะไร,"{'answer': ['ราบัต'], 'answer_begin_position':..."
3998,3999,355975,"<doc id=""355975"" url=""https://th.wikipedia.org...",กลุ่มบริษัท คิง เพาเวอร์ เป็นบริษัทด้านธุรกิจค...,"{'answer': ['2532'], 'answer_begin_position': ..."


In [11]:
q1["context_text"] = q1["context"].apply(extract_text)
q2["context_text"] = q2["context"].apply(extract_text)

In [ ]:
# qa_train.head()

,question_id,article_id,title,context,question,answers
0,01KZTrxgvC5mOovXFMPJ_000,01KZTrxgvC5mOovXFMPJ,พัทธ์ธีรา ศรุติพงศ์โภคิน,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร,"{'answer_end': [50], 'answer_start': [31], 'te..."
1,01KZTrxgvC5mOovXFMPJ_001,01KZTrxgvC5mOovXFMPJ,พัทธ์ธีรา ศรุติพงศ์โภคิน,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร,"{'answer_end': [70], 'answer_start': [68], 'te..."
2,01KZTrxgvC5mOovXFMPJ_002,01KZTrxgvC5mOovXFMPJ,พัทธ์ธีรา ศรุติพงศ์โภคิน,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร,"{'answer_end': [92], 'answer_start': [75], 'te..."
3,01KZTrxgvC5mOovXFMPJ_003,01KZTrxgvC5mOovXFMPJ,พัทธ์ธีรา ศรุติพงศ์โภคิน,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร,"{'answer_end': [155], 'answer_start': [139], '..."
4,01asKzWKkocCMjZAITfL_001,01asKzWKkocCMjZAITfL,คลีโอพัตรา,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,บิดาของคลีโอพัตราเป็นใคร,"{'answer_end': [339], 'answer_start': [319], '..."


In [14]:
from pythainlp.tokenize import word_tokenize
q1["context_text_wordsegged"] = q1["context_text"].apply(lambda x: ' '.join(word_tokenize(x.replace(' ',''))))
q2["context_text_wordsegged"] = q2["context_text"].apply(lambda x: ' '.join(word_tokenize(x.replace(' ',''))))
# q3["context_text_wordsegged"] = q3["context"].apply(lambda x: ' '.join(word_tokenize(x)))

In [15]:
q1["context_text_wordsegged"][0]

"เบน จี้ เบน จี้ () เป็น ชื่อ ตัวละคร หมา พันทาง แสนรู้ ที่ ปรากฏ อยู่ ใน ภาพยนตร์ หลาย เรื่อง ที่ เขียน บท และ กำกับ โดย โจ แคมป์ ใน ช่วง ทศวรรษ 1970 ถึง 1980 ภาพยนตร์ เรื่อง แรก ใน ชุด ใช้ ชื่อเรื่อง ว่า เบน จี้ เช่นเดียวกับ ตัวละคร ถ่ายทำ ที่ เมือง ดัลลัส รัฐ เทกซัส ฉาย ครั้งแรก ใน ปี พ.ศ. 2517 ภาพยนตร์ ได้รับ การ เสนอชื่อ เข้า ชิง รางวัล ออสการ์ และ ได้ รางวัล ลูกโลก ทองคำ สาขา เพลง ประกอบ ยอดเยี่ยม จาก เพลง Benji ' sTheme ( IFeelLove ) ร้อง โดย ชาร์ลี ริ ช หมา ที่ แสดง เป็น เบน จี้ ตัว แรก ชื่อว่า ฮิ กกิ้นส์( พ.ศ. 2502 - พ.ศ. 2518 ) มีอายุ ถึง 15 ปี แล้ว ใน ขณะ แสดง หลังจาก ภาพยนตร์ ออก ฉาย ได้ ไม่ นาน มัน ก็ ตาย ใน ปี พ.ศ. 2518 เบน จี้ ใน ภาพยนตร์ เบน จี้ ใน ภาพยนตร์ . - พ.ศ. 2517 , Benji ( ภาพยนตร์ ) - พ.ศ. 2520 , FortheLoveofBenji ( ภาพยนตร์ ) - พ.ศ. 2521 , Benji ' sVeryOwnChristmasStory ( ภาพยนตร์โทรทัศน์ ) - พ.ศ. 2523 , OhHeavenlyDog ( ภาพยนตร์ ) - พ.ศ. 2523 , BenjiatWork ( ภาพยนตร์โทรทัศน์ ) - พ.ศ. 2524 , BenjiTakesaDiveatMarineland ( ภาพยนตร์โทรทัศน์ ) - พ.ศ. 2526 , Benji ,

In [16]:
q1["question_wordsegged"] = q1["question"].apply(lambda x: ' '.join(word_tokenize(x.replace(' ',''))))
q2["question_wordsegged"] = q2["question"].apply(lambda x: ' '.join(word_tokenize(x.replace(' ',''))))
# q3["question_wordsegged"] = q3["question"].apply(lambda x: ' '.join(word_tokenize(x)))

In [ ]:
q1["question_wordsegged"]

In [18]:
q1["answers_wordsegged"] =  q1["answers"].apply(lambda x: ' '.join(word_tokenize(x["answer"][0].replace(' ',''))))
q2["answers_wordsegged"] =  q2["answers"].apply(lambda x: ' '.join(word_tokenize(x["answer"][0].replace(' ',''))))
# q3["answers_wordsegged"] =  q3["answers"].apply(lambda x: ' '.join(word_tokenize(x["answer"][0])))

In [ ]:
q1["answers_wordsegged"]

In [21]:
answer_starts = []
for idx, row in q1.iterrows():
  try:
    answer_start = row["context_text_wordsegged"].index(row["answers_wordsegged"])
  except:
    answer_start = -1
  answer_starts.append(answer_start)

In [22]:
q1["answer_starts"] = answer_starts

In [23]:
answer_starts = []
for idx, row in q2.iterrows():
  try:
    answer_start = row["context_text_wordsegged"].index(row["answers_wordsegged"])
  except:
    answer_start = -1
  answer_starts.append(answer_start)

In [24]:
q2["answer_starts"] = answer_starts

In [25]:
train_data = []
for idx, row in q1.iterrows():
  doc = {}
  doc["context"] = row["context_text_wordsegged"]
  doc["qas"] = [{"id":idx, "is_impossible":row["answer_starts"]<0,"question":row["question_wordsegged"],"answers":[{"text":row["answers_wordsegged"], "answer_start":row["answer_starts"]}]}]
  train_data.append(doc)

In [ ]:
for idx, row in q2.iterrows():
  doc = {}
  doc["context"] = row["context_text_wordsegged"]
  doc["qas"] = [{"id":idx, "is_impossible":row["answer_starts"]<0,"question":row["question_wordsegged"],"answers":[{"text":row["answers_wordsegged"], "answer_start":row["answer_starts"]}]}]
  train_data.append(doc)

In [26]:
test_data = []
for idx, row in q2.iterrows():
  doc = {}
  doc["context"] = row["context_text_wordsegged"]
  doc["qas"] = [{"id":idx, "is_impossible":row["answer_starts"]<0,"question":row["question_wordsegged"],"answers":[{"text":row["answers_wordsegged"], "answer_start":row["answer_starts"]}]}]
  test_data.append(doc)

In [27]:
len(test_data)

74

In [28]:
len(train_data)

4000

In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel,QuestionAnsweringArgs

# Configure the model
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 32
model_args.evaluate_during_training = True
model_args.num_train_epochs = 7
model_args.overwrite_output_dir = True

# model_args = {
#     "reprocess_input_data": True,
#     'train_batch_size':32,
#     "use_early_stopping": True,
#     "early_stopping_delta": 0.01,
#     "early_stopping_metric": "mcc",
#     "early_stopping_metric_minimize": False,
#     "early_stopping_patience": 5,
#     "evaluate_during_training_steps": 500,
#     "fp16": False,
#     "num_train_epochs":3,
#     "overwrite_output_dir":True
# }

Bert = QuestionAnsweringModel(
    "xlmroberta", "/content/drive/MyDrive/TheBeyon-xlm/best_model", args=model_args
)

# Train the model
Bert.train_model(train_data, eval_data=train_data)

# Evaluate the model
result, texts = Bert.eval_model(test_data)

add example index and unique id: 100%|██████████| 4000/4000 [00:00<00:00, 263184.40it/s]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 0 of 7:   0%|          | 0/609 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 4000/4000 [01:21<00:00, 49.29it/s]

add example index and unique id: 100%|██████████| 4000/4000 [00:00<00:00, 224204.41it/s]


Running Evaluation:   0%|          | 0/2433 [00:00<?, ?it/s]

Running Epoch 1 of 7:   0%|          | 0/609 [00:00<?, ?it/s]

In [ ]:
result, texts = Bert.eval_model(train_data)

In [ ]:
result

In [ ]:
texts 

In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel


In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel
xlmroberta = QuestionAnsweringModel(
    "bert", "/content/outputs/best_model")

In [ ]:
!cp -av /content/outputs/best_model /content/drive/MyDrive/TheBeyon-xlm